# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770497663059                   -0.53    9.0         
  2   -2.771692112917       -2.92       -1.32    1.0    176ms
  3   -2.771714514194       -4.65       -2.53    1.0    120ms
  4   -2.771714675882       -6.79       -3.38    1.0    122ms
  5   -2.771714713365       -7.43       -3.90    2.0    177ms
  6   -2.771714715219       -8.73       -5.06    1.0    124ms
  7   -2.771714715246      -10.57       -5.23    2.0    153ms
  8   -2.771714715249      -11.41       -6.10    1.0    132ms
  9   -2.771714715250      -12.76       -6.51    2.0    161ms
 10   -2.771714715250      -13.80       -7.67    1.0    139ms
 11   -2.771714715250   +  -14.05       -8.30    2.0    158ms


-0.00013457412066031514

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770508054156                   -0.53    8.0         
  2   -2.771773932024       -2.90       -1.32    1.0    192ms
  3   -2.771801465921       -4.56       -2.53    1.0    122ms
  4   -2.771802048807       -6.23       -3.42    2.0    158ms
  5   -2.771802072522       -7.62       -3.89    1.0    153ms
  6   -2.771802074262       -8.76       -4.65    1.0    125ms
  7   -2.771802074473       -9.68       -5.18    2.0    171ms
  8   -2.771802074476      -11.51       -6.22    1.0    132ms
  9   -2.771802074476      -12.89       -6.98    2.0    163ms
 10   -2.771802074476   +  -14.65       -8.33    1.0    141ms


0.017612221548308864

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457412066031514
Displaced dipole:  0.017612221548308864
Polarizability :   1.7746795668969177


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920985821e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766552574671e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852745451645e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694605739974e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088782530715e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.284928366628e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.045623899371e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.045623899371e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.004633686686e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 7.093083878741e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.055123479535e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.055123479535e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.0551190442840076e-12
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.